In [44]:
import pandas as pd 
import numpy as np 
import math
from datetime import datetime
import warnings 
warnings.filterwarnings('ignore')
from flask import Flask, jsonify
import statistics

def read_data(path='data.csv'): 
    df = pd.read_csv(path, header=None)
    columns = ['IdHdr', 'IdClient', 'ClientName', 'Amount', 'DscItem', 'IdItem', 'Date', 'ItemType', 'IdItemType','min']
    df.columns = columns
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d %H:%M:%S.%f')
    df['Date'] = df["Date"].dt.date
    # sort values base on date column 
    df = df.sort_values(by='Date')
    df = df.drop_duplicates()
    return df


def make_period_time(df: pd.DataFrame, days:int=61):
    # now = datetime.today().date()
    now = pd.to_datetime('2023-11-24')
    now = now.date()
    df['Recently'] = now - df['Date']
    
    def make_true(column):
        if column.days > days:
            return False
        else:
            return True
    
    df['Assess'] = df['Recently'].apply(make_true)
    df1 = df[df['Assess'] == True]
    return df1 


def calculate_max_orders(period_df: pd.DataFrame, std_criterion:float=2):
    list_names = period_df['ClientName'].unique()
    list_items = period_df['DscItem'].unique()
    
    period_df['Mean'] = 0.
    period_df['Std'] = 0.
    for name in list_names[0:]:
        for item in list_items[0:]:
            if item in (period_df[period_df['ClientName'] == name]['DscItem'].unique()):
                items = period_df[(period_df['ClientName']==name) & (period_df['DscItem']==item)]['Amount']
                mean = items.mean()
                try:
                    # Calculate sum of squared differences from the mean
                    squared_diffs = sum((x - mean) ** 2 for x in items)
                    # Calculate population standard deviation
                    std = (squared_diffs / len(items)) ** 0.5
                except:
                    std = 0
                period_df.loc[(period_df['ClientName'] == name) & (period_df['DscItem'] == item), 'Mean'] = mean
                period_df.loc[(period_df['ClientName'] == name) & (period_df['DscItem'] == item), 'Std'] = std
    
    period_df['Std'] = period_df['Std'].fillna(0)
    period_df['StdCriterion'] = std_criterion
    period_df['Max'] = (period_df['Std'] * period_df['StdCriterion']) + period_df['Mean']
    period_df['Max'] = period_df['Max'].apply(lambda x: math.ceil(x))
    
    return period_df


def find_max(id_client:int=0, id_item:int=0):
    df = read_data()
    period_df = make_period_time(df)
    last_df = calculate_max_orders(period_df, std_criterion=2.0)
    # max_value = last_df[(last_df['IdClient']==id_client) & (last_df['IdItem']==id_item)]['Max']
    # max_value = max_value.iloc[-1]
    max_df = last_df[['IdClient', 'IdItem', 'IdItemType', 'min', 'Mean', 'Std', 'StdCriterion', 'Max']]
    max_df = max_df.drop_duplicates()
    # max_df.sort_values(by=max_df['IdClient'])
    return max_df


In [45]:
df = read_data()
df

,IdHdr,IdClient,ClientName,Amount,DscItem,IdItem,Date,ItemType,IdItemType,min
0,37201,21697122,امید سیب عباس اباد فکری,5.00,سالاد مرغ گریل م,1063201,2023-04-09,روزانه,201,1.00
1,37201,21697122,امید سیب عباس اباد فکری,2.00,سالاد اسفناج - م,1063001,2023-04-09,روزانه,201,1.00
2,37201,21697122,امید سیب عباس اباد فکری,5.00,سالاد سبز,1063101,2023-04-09,روزانه,201,1.00
3,37201,21697122,امید سیب عباس اباد فکری,4.00,کاهو پیچ فرش گاردن,1063301,2023-04-09,روزانه,201,1.00
4,37201,21697122,امید سیب عباس اباد فکری,2.00,بیبی اسفناج فرش گاردن,1062901,2023-04-09,روزانه,201,1.00
...,...,...,...,...,...,...,...,...,...,...
94337,5303701,21700722,سیب ستارخان حمید رضوی زاده,5.05,سوسیس کوکتل انگشتی,1059901,2023-11-23,یخچالی,1,1.00
94338,5303701,21700722,سیب ستارخان حمید رضوی زاده,10.50,پپرونی مکزیکی,1059801,2023-11-23,یخچالی,1,0.85
94339,5303801,21740822,شیراز جنوبی - وحید رضوی زاده,5.00,سالاد اسکاندیناوی م,1056901,2023-11-23,روزانه,201,1.00
94341,5303801,21740822,شیراز جنوبی - وحید رضوی زاده,10.00,سالاد سزار م,1057001,2023-11-23,روزانه,201,1.00


In [46]:
df[(df['IdClient'] == 21761222) & (df['IdItem'] == 1061001)]

,IdHdr,IdClient,ClientName,Amount,DscItem,IdItem,Date,ItemType,IdItemType,min
87062,5144901,21761222,رستاک آگور شعبه امام حسین,50.0,پنیر پیتزا 202,1061001,2023-11-06,یخچالی,1,10.0
88633,5180301,21761222,رستاک آگور شعبه امام حسین,20.0,پنیر پیتزا 202,1061001,2023-11-09,یخچالی,1,10.0
92073,5257101,21761222,رستاک آگور شعبه امام حسین,50.0,پنیر پیتزا 202,1061001,2023-11-18,یخچالی,1,10.0
94290,5303301,21761222,رستاک آگور شعبه امام حسین,30.0,پنیر پیتزا 202,1061001,2023-11-22,یخچالی,1,10.0


In [18]:
df = make_period_time(df=df)
df[(df['IdClient'] == 21761222) & (df['IdItem'] == 1061001)]

,IdHdr,IdClient,ClientName,Amount,DscItem,IdItem,Date,ItemType,IdItemType,min,Recently,Assess
87062,5144901,21761222,رستاک آگور شعبه امام حسین,50.0,پنیر پیتزا 202,1061001,2023-11-06,یخچالی,1,10.0,18 days,True
88633,5180301,21761222,رستاک آگور شعبه امام حسین,20.0,پنیر پیتزا 202,1061001,2023-11-09,یخچالی,1,10.0,15 days,True
92073,5257101,21761222,رستاک آگور شعبه امام حسین,50.0,پنیر پیتزا 202,1061001,2023-11-18,یخچالی,1,10.0,6 days,True
94290,5303301,21761222,رستاک آگور شعبه امام حسین,30.0,پنیر پیتزا 202,1061001,2023-11-22,یخچالی,1,10.0,2 days,True


In [27]:
list_names = df['ClientName'].unique()
for i, name in enumerate(list_names):
    if name == "رستاک آگور شعبه امام حسین":
        print(i, name)

61 رستاک آگور شعبه امام حسین


In [30]:
list_items = df['DscItem'].unique()
for item in list_items:
    if item == "پنیر پیتزا 202":
        print(item)

پنیر پیتزا 202


In [47]:
max_df = find_max()
print(max_df[(max_df['IdClient']==21761222) & (max_df['IdItem'] == 1061001)])

       IdClient   IdItem  IdItemType   min  Mean        Std  StdCriterion  Max
87062  21761222  1061001           1  10.0  37.5  12.990381           2.0   64


In [43]:
numbers = [50, 20, 50, 30]

# Calculate mean
mean = statistics.mean(numbers)

# Calculate sum of squared differences from the mean
squared_diffs = sum((x - mean) ** 2 for x in numbers)

# Calculate population standard deviation
pop_std_dev = (squared_diffs / len(numbers)) ** 0.5

print("Population Standard Deviation:", pop_std_dev)

Population Standard Deviation: 12.99038105676658
